#### Lien vers le site

https://medium.com/jatana/unsupervised-text-summarization-using-sentence-embeddings-adb15ce83db1

# Premier summarizer

Je vais commencer par faire un premier type de summarizer, simple, basique, peu efficace, dans le but d'avoir une première base de travail. 

Ce dernier se décomposera en plusieurs étapes (7 précisément) et chacune des étapes fera office de partie au sein de ce notebook.

## Importation du'un modèle pour la langue francaise

In [2]:
import spacy
path = "C:\\Users\\ivaro\\OneDrive\\Bureau\\NLP\\" #A modifier/automatiser
nlp = spacy.load(path + "fr_core_news_sm-2.2.5") #Charge le modele fr deja installe

## Lecture du texte

In [3]:
with open('C:\\Users\\ivaro\\OneDrive\\Bureau\\Stage_3A\\Debat\\debat_2007.txt', 'r', encoding = "utf-8") as file:
     texte = file.read()
        
with open('C:\\Users\\ivaro\\OneDrive\\Bureau\\Stage_3A\\Debat\\Interview_Prelats.txt', 'r', encoding = "utf-8") as file:
     prelat = file.read()        

## Tokenisation

Maintenant que j'ai récupéré le texte à résumer, je vais chercher à déterminer l'importance d'une phrase pour cela je commence par tokeniser mon texte phrase par phrase.

In [4]:
def return_token_sent(texte):
    """Tokenisation du texte par phrase"""
    doc = nlp(texte)
    # Retourner le texte de chaque phrase
    return [X.text for X in doc.sents]

Tokenize = return_token_sent(texte)
len(Tokenize)

405

On a bien une séparation par phrase excepté au début, avec le Bonsoir. 

## Vectorisation 

### Vectorisation mot par mot

In [4]:
import numpy as np

def return_word_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le vecteur lié à chaque token
    return [(X.vector) for X in doc]

Sentence_emb0 = return_word_embedding(Tokenize[0])

Pour la première phrase on constate 7 mots dont le point et les deux points. A supprimer ainsi que les stopwords par la suite.

### Vectorisation par phrase

On va "moyenner" les vecteurs d'une meme phrase pour pouvoir avoir une représentation vectorielle d'une phrase

In [5]:
def return_sentence_emb(sentence):
    """Prend en entree une phrase vectorise mot a mot"""
    somme = sentence[0]
    for i in range(len(sentence)) :
        if i != 0 :
            somme += sentence[i]
    return somme/len(sentence)

Vect = return_sentence_emb(Sentence_emb0)

In [19]:
def return_text_emb(texte) : 
    """Vectorisation du texte"""
    text_vect = []
    Tokenize = return_token_sent(texte) #Tokenisation par phrase    
    for i in range(len(Tokenize)) :
        Sentence_emb = return_word_embedding(Tokenize[i]) #Vectorisation des phrases
        vect = return_sentence_emb(Sentence_emb)
        text_vect += [vect]
    return text_vect

#Test = return_text_emb(texte)

## Clusterisation

Maintenant on va effectuer le Kmeans. Tout d'abord on va choisir le nombre de centre. Dans notre configuration on va avoir, un centre = une phrase. Comme on souhaite faire un résumé, on peut choisir de conserver X% du texte initial. Par exemple 10%. C'est ce nombre que l'on va conserver.

In [20]:
import numpy as np
from sklearn.cluster import KMeans

nb_phrase = len(Tokenize)

nb_centre = int(np.ceil(0.10*nb_phrase)) 
#partie entiere avec np.ceil qui renvois un flottant -> int pour le kmeans
kmeans = KMeans(n_clusters=nb_centre)
kmeans = kmeans.fit(Test)

KeyboardInterrupt: 

In [28]:
#print(kmeans.labels_) #Pour avoir le numero du cluster attribue

In [25]:
from sklearn.metrics import pairwise_distances_argmin_min

avg = []
for j in range(nb_centre):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, Test)
ordering = sorted(range(nb_centre), key=lambda k: avg[k])
summary = ' '.join([Tokenize[closest[idx]] for idx in ordering])

summary

"Chacun l'a observé. 25 ans. 45 %! Bonsoir Ségolène Royal, bonsoir, Nicolas Sarkozy. Ségolène Royal: Sur l'hôpital public, qui est une question cruciale, essentielle, puisque c'est aussi sur le maintien des petits hôpitaux, la présence de santé sur l'ensemble du territoire national. Si vous augmentez les dépenses, vous augmentez les impôts. Je propose une chose: 45 % du budget de la France, c'est le salaire de la fonction publique et les pensions de retraite. Voulez-vous aussi dire que vous allez supprimer des poste de magistrats alors qu'il n'y a aucune réponse pour les jeunes délinquants...\n\nNicolas Sarkozy: Mme Royal dit qu'elle va transférer aux régions, ce qui leur permettra de faire une autre augmentation après les autres augmentations faramineuses que vous avez faites les années passées. Il y aura donc, si je suis président de la République, je veux proposer aux fonctionnaires un pacte de progrès. Si la croissance est supérieure à 2,5%, à la fois, je finance mon pacte présiden

Clairement à améliorer.

- Le decoupage/tokenisation n'est pas parfait et loin de la. 
- On peut supprimer la ponctuation au sein d'une phrase car elle n'apporte rien et rentre en compte par la suite dans l'embedding ? Mauvaise idée car la ponctuation peut finalement etre utile.
- On peut supprimer les stopwords qui n'apportent pas d'information ou peu.
- Ici on a affaire à une interview/debat. On peut donc localiser les entités nommées. (Utile pour le format réunion). 
- Localisation des différents sujets pour résumer par sujet. 
- La méthode d'embedding des phrases est brute et surement pas adaptée. Pensez à pondérer selon l'importance du mot ?
- Summarizer ? pas ouf....
- pk une dimension 32 dans l'embeding des mots
- ajouter les formules de politesse aux stopwords / les retirer dans tous les cas car n'apportent pas de sens


# Améliorations

### Sur la Tokenisation

In [37]:
## Seconde methode pour Tokeniser. 
import nltk
from nltk.tokenize import sent_tokenize
#nltk.download("punkt")
Tokenize = sent_tokenize(prelat, language = "french")
Tokenize
#len(Tokenize)

['Pour ce nouveau PotterAfter, nous vous proposons de découvrir une saga littéraire entièrement financée via un site de financement participatif.',
 'Aujourd’hui, je viens vous parler d’une saga d’heroic fantasy découverte lors de l’édition 2019 du Salon du Livre de Paris, Les Prélats de Faneas de Charlotte Abécassis Weigel.',
 'Elle se compose pour l’instant de trois romans, Les terres d’exil, Le soulèvement des fiefs et L’alliance d’Amiran, tous publiés grâce à un site de financement participatif.',
 'J’ai d’abord été attirée par la couverture très sobre du premier tome, un petit renard stylisé sur fond bleu, puis une fois le livre en main, j’ai tout de suite été captivée par les trois premières phrases :\n\n«Vous qui n’avez ni dieu, ni pouvoir.',
 'Vous qui avez abandonné vos rêves, les prenant pour des chimères.',
 'Ne vous êtes-vous jamais demandé où naissent les récits de vos anciennes légendes?»\n\nL’HISTOIRE\nNous suivons les aventures d’Astéria et de sa cousine Cléora, envoyée

La tokenisation semble de meilleurs qualités !

## Suppression des StopWords

In [ ]:
#Pour supprimer les stopWords 
stopWords = set(stopwords.words('french'))

def delete_SW(test,stopWords) :
    clean_words = []
    for token in return_token(test):
        if token not in stopWords:
            clean_words.append(token)
    return clean_words

clean_words = delete_SW(test)
clean_words

#Ajout d'elements a SW

def add_SW(L,SW) :
    """Ajoute une liste d element"""
    for j in L:
       SW.add(j) 
    return SW

In [33]:
#Relancer la fonction de resume avec les cgts

def return_text_emb(texte) : 
    """Vectorisation du texte"""
    text_vect = []
    Tokenize = sent_tokenize(texte, language = "french") #Tokenisation par phrase    
    for i in range(len(Tokenize)) :
        Sentence_emb = return_word_embedding(Tokenize[i]) #Vectorisation des phrases
        vect = return_sentence_emb(Sentence_emb)
        text_vect += [vect]
    return text_vect

Test = return_text_emb(prelat)

In [36]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

nb_phrase = len(Tokenize)

nb_centre = int(np.ceil(0.15*nb_phrase)) 
#partie entiere avec np.ceil qui renvois un flottant -> int pour le kmeans
kmeans = KMeans(n_clusters=nb_centre)
kmeans = kmeans.fit(Test)

avg = []
for j in range(nb_centre):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, Test)
ordering = sorted(range(nb_centre), key=lambda k: avg[k])
summary = ' '.join([Tokenize[closest[idx]] for idx in ordering])

summary

'POURQUOI VOUS PROPOSER CES ROMANS ? L’INFLUENCE DE L’OEUVRE DE J. K. ROWLING\nL’autrice le dit elle-même sur son site, «par certains aspects, cet univers pourrait plaire à ceux qui auraient aimé Harry Potter, Le Seigneur des anneaux, Twilight, Eragon… et beaucoup d’autres.» Car en plus de nous plonger dans un univers fantastique qui cache de multiples secrets, les livres de Charlotte Abécassis Weigel sont empreints de magie, de suspens, de trahisons… Et quoi de mieux pour avoir des informations sur un roman que d’avoir les explications de son autrice ? Elles n’ont pas été envoyées dans notre monde par hasard, elles ont une mission : trouver les deux Prélats disparus, ceux de la terre et du feu. J’ai d’abord été attirée par la couverture très sobre du premier tome, un petit renard stylisé sur fond bleu, puis une fois le livre en main, j’ai tout de suite été captivée par les trois premières phrases :\n\n«Vous qui n’avez ni dieu, ni pouvoir. Premièrement, je n’avais pas très envie de pas

## Pondération de la phrase

On va chercher a calculer la frequence d'apparition de chaque mot dans notre texte pour donner de l importance aux mots "rares"

### Obtention de la fréquence



In [6]:
#Récupération des racines de mots

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')

def return_stem(sentence):
    doc = nlp(sentence)
    return [stemmer.stem(X.text) for X in doc] #X.text correspond au mot



In [7]:
def obt_occ(dictionnaire,stopwords,mot) :
    """Renvoie le nombre d'occurence du mot """
    if mot in stopwords :
        return 0
    else : 
        if (mot in dictionnaire) :
            return 1/dictionnaire[mot]
        return 1

In [9]:
#Fonctionne, mais doit tourner sur plusieurs textes pour etre efficace ?
def word_in_text(texte):
    """Renvoie un dictionnaire avec en cle
    les mots du texte et en valeur le nombre d'occurence"""
    dictionnaire = {}
    tokens = return_stem(texte) #Recuperation des racines des tokens
    n = len(tokens)
    for i in range(len(tokens)): #Pour tout les tokens 
        if (tokens[i] in dictionnaire):
            dictionnaire[tokens[i]] = dictionnaire[tokens[i]] + (1/n)
        else:
            dictionnaire[tokens[i]] = (1/n)
    return dictionnaire

occ_tot = word_in_text(prelat)
occ_tot

{'pour': 0.006302521008403361,
 'ce': 0.004726890756302521,
 'nouveau': 0.0010504201680672268,
 'potteraft': 0.0005252100840336134,
 ',': 0.05987394957983188,
 'nous': 0.007352941176470587,
 'vous': 0.005777310924369747,
 'proposon': 0.0005252100840336134,
 'de': 0.05252100840336132,
 'découvr': 0.002626050420168067,
 'une': 0.009453781512605041,
 'sag': 0.0021008403361344537,
 'littérair': 0.0005252100840336134,
 'entier': 0.0005252100840336134,
 'financ': 0.0031512605042016803,
 'vi': 0.002626050420168067,
 'un': 0.012605042016806721,
 'sit': 0.0015756302521008404,
 'particip': 0.0036764705882352936,
 '.': 0.031512605042016854,
 '\n\n\n': 0.0005252100840336134,
 'aujourd’hui': 0.0010504201680672268,
 'je': 0.009453781512605041,
 'vien': 0.0005252100840336134,
 'parl': 0.0005252100840336134,
 'd’': 0.015231092436974788,
 'heroic': 0.0005252100840336134,
 'fantasy': 0.0010504201680672268,
 'découvert': 0.0010504201680672268,
 'lor': 0.0005252100840336134,
 'l’': 0.013655462184873948,
 

In [ ]:
#Fonction initiale
def return_sentence_emb(sentence):
    """Prend en entree une phrase vectorise mot a mot"""
    somme = sentence[0]
    for i in range(len(sentence)) :
        if i != 0 :
            somme += sentence[i]
    return somme/len(sentence)

Vect = return_sentence_emb(Sentence_emb0)